### Build the Chroma DB

In [1]:
# import chromadb and create client
import chromadb
import pandas as pd

In [3]:
file_data = pd.read_csv('imdb_top_1000.csv')
client = chromadb.Client()

In [5]:
file_data.head(1)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"


In [24]:
# get the data from file_data and create chromadb collection
Overviews = []
metadatas = []
ids = []

for index, row in file_data.iterrows():
    Overviews.append(row['Overview'])
    metadatas.append({
                     "Title": row['Series_Title'],
                     "Poster Links": row['Poster_Link'],
                     "Released_Year":row['Released_Year'],
                     "Genre":row['Genre'],
                     'Overview': row['Overview'],
                     "No. of Votes" : row['No_of_Votes'],
                     "Views" : row['Gross']})
    ids.append(str(index + 1))

In [26]:
# Check if the collection exists and delete it
existing_collections = client.list_collections()
if 'imbd_collection' in [collection.name for collection in existing_collections]:
    client.delete_collection('imbd_collection')

In [28]:
print(client.list_collections())

[]


In [30]:
# create collection of imbd files 
imbd_collection = client.create_collection("imbd_collection")

# add files to the chromadb collection
imbd_collection.add(
    documents=Overviews,
    metadatas=metadatas,
    ids=ids
)

In [42]:
# query the database to get the answer from vectorized data
results = imbd_collection.query(
    query_texts=["What is the most funny romantic movie?"], 
    n_results=2
)

results

{'ids': [['755', '790']],
 'distances': [[0.7472079396247864, 1.1919960975646973]],
 'metadatas': [[{'Genre': 'Comedy, Drama, Romance',
    'No. of Votes': 472242,
    'Overview': "An offbeat romantic comedy about a woman who doesn't believe true love exists, and the young man who falls for her.",
    'Poster Links': 'https://m.media-amazon.com/images/M/MV5BMTk5MjM4OTU1OV5BMl5BanBnXkFtZTcwODkzNDIzMw@@._V1_UX67_CR0,0,67,98_AL_.jpg',
    'Released_Year': '2009',
    'Title': '(500) Days of Summer',
    'Views': '32,391,374'},
   {'Genre': 'Comedy, Drama',
    'No. of Votes': 178565,
    'Overview': "A lovelorn screenwriter becomes desperate as he tries and fails to adapt 'The Orchid Thief' by Susan Orlean for the screen.",
    'Poster Links': 'https://m.media-amazon.com/images/M/MV5BZjIwZWU0ZDItNzBlNS00MDIwLWFlZjctZTJjODdjZWYxNzczL2ltYWdlXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg',
    'Released_Year': '2002',
    'Title': 'Adaptation.',
    'Views': '22,245,861'}]],
 'emb